In [1]:
import data_utils as du
import pandas as pd
import networkx as nx

data_dir = du.find_data_dir('app')
dataloader_file = du.get_file_path(data_dir, 'class based structure', 'dataloaders', 'dataloader.p')
dataloader = du.read_from_pickle(dataloader_file)

In [35]:
dataloader.target_distribution_stats()

,Amount,Percentage
-1,91,39.9%
0,70,30.7%
1,67,29.4%
Total,228,100.0%


### Experimentation with ML models

In [28]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.metrics import f1_score, SCORERS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [49]:
h = .02  # step size in the mesh

names = ["Nearest Neighbors 2", "Nearest Neighbors 4", "Linear SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes"]

classifiers = [
    KNeighborsClassifier(2),
    KNeighborsClassifier(4),
    SVC(kernel="linear", C=0.025),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=100, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB()
]

X, y = dataloader.X.values, np.array(dataloader.y).astype(int)

# X = StandardScaler(with_mean=False).fit_transform(X)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.5, random_state=43)

scores = []

# iterate over classifiers
for name, clf in zip(names, classifiers):
    print(name)
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    f1_macro = f1_score(y_test, predicted, average='macro')
    f1_micro = f1_score(y_test, predicted, average='micro')
    scores.append((name, f1_macro, f1_micro))
    
df = pd.DataFrame(scores, columns=['Name', 'f1_macro', 'f1_micro']).set_index('Name', drop=True).style.format({'f1_macro': "{:.1%}", 'f1_micro': "{:.1%}"})
df

Nearest Neighbors 2
Nearest Neighbors 4
Linear SVM
Gaussian Process
Decision Tree
Random Forest
Neural Net
AdaBoost
Naive Bayes


,f1_macro,f1_micro
Name,,
Nearest Neighbors 2,39.3%,46.5%
Nearest Neighbors 4,38.9%,43.0%
Linear SVM,33.8%,34.2%
Gaussian Process,35.2%,43.9%
Decision Tree,39.9%,44.7%
Random Forest,37.6%,39.5%
Neural Net,38.8%,41.2%
AdaBoost,36.1%,39.5%
Naive Bayes,33.2%,34.2%
